# Transforming data from Wikipedia page to dataframe

In [1]:
import pandas as pd

In [2]:
#Using pandas to read Wikipedia table into a dataframe
post_codes=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

Next the post codes with unassigned boroughs were removed and unassigned neighborhoods were renamed with their borough name

In [3]:
post_codes=post_codes[post_codes.Borough!='Not assigned'].reset_index(drop=True)
for hoods in post_codes.Neighborhood:
    if 'Neighborhood'=='Not assigned':
        post_codes['Neighborhood']=post_codes['Borough']
post_codes.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Adding latitude and longitude coordinates of each neighborhood

In [ ]:
#!pip install geocoder
import geocoder
Lat_list=[]
Lng_list=[]
for i in range(post_codes.shape[0]):
    address='{}, Toronto, Ontario'.format(post_codes.at[i,'Postal Code'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

     |████████████████████████████████| 102kB 8.8MB/s ta 0:00:011


In [ ]:
toronto_merged = post_codes
toronto_merged['Latitude']=Lat_list
toronto_merged['Longitude']=Lng_list
toronto_merged.head(12)

In [ ]:
toronto_merged.shape

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

In [ ]:
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_merged

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 